In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
from scipy.interpolate import NearestNDInterpolator
from math import * 
plt.rcParams['figure.figsize'] = [12, 9]



In [ ]:
img_1 = cv2.imread("./inputs/P3/sonderkommando_photographs_280.png",0)
img_2 = cv2.imread("./inputs/P3/sonderkommando_photographs_281.png",0)

img_1_points = np.array([[319, 406], [138, 406], [138, 159],[319, 111]], dtype=np.float32)
img_2_points = np.array([[331, 380],[174, 370] , [158,100], [324, 100]], dtype=np.float32)

In [ ]:
def GeoTransform(img,orginal_points):
    window = 6

    selected_p_img = img.copy()
    for point in orginal_points:
        point = point.astype(np.int32)
        cv2.rectangle(selected_p_img,(point[0]-window,point[1]-window),
                         (point[0]+window,point[1]+window),(255),3)
        
    new_w = int((np.linalg.norm((orginal_points[0]+orginal_points[3])/2 - (orginal_points[1]+orginal_points[2])/2))*1.3)
    new_h = int(np.linalg.norm((orginal_points[0]+orginal_points[1])/2 - (orginal_points[2]+orginal_points[3])/2))


    out_put_points = np.array([[0, 0], [new_w, 0], [new_w, new_h], [0, new_h]], dtype=np.float32)

    Transformer= cv2.getPerspectiveTransform(orginal_points, out_put_points)
    print(Transformer)
    out_put_img = cv2.warpPerspective(img, Transformer, (new_w, new_h))
    out_put_img = cv2.rotate(out_put_img,cv2.ROTATE_180)
    return selected_p_img,out_put_img


def gammaCorrection(src, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv2.LUT(src, table)

def Enhance_2(out):
    
    out = out.copy()
    img = cv2.fastNlMeansDenoising(out,h=11)
    img_out_1 = img.copy()
    
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    img = cv2.filter2D(img, -1, sharpen_kernel)
    img_out_2 = img.copy()
    
    img = gammaCorrection(img,1.7)
    
    return img_out_1,img_out_2,img


def handle(_in:int):
    if _in ==1 :
        img = img_1
        pnts = img_1_points
    else:
        img = img_2
        pnts = img_2_points
        
    selec,out = GeoTransform(img,pnts)
    cont,shar,enhance = Enhance_2(out)
    
    results = [img,selec,out,cont,shar,enhance]
    tags = ["Orginal Input","Selected points to transform","Geometric Transformations","Denoise with NlMean","Deblur with laplacian","Gamma Correction"]
    plt.rcParams['figure.figsize'] = [12, 9]
    for index in range(6):
        plt.subplot(2,3,index+1)
        plt.imshow(results[index],cmap="gray")
        plt.axis("off")
        plt.title(tags[index])
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [8, 6]
    plt.subplot(1,2,1)
    plt.imshow(results[0],cmap="gray")
    plt.axis("off")
    plt.title(tags[0])
    
    plt.subplot(1,2,2)
    plt.imshow(results[-1],cmap="gray")
    plt.axis("off")
    plt.title(tags[-1])
    
    plt.tight_layout()
    plt.show()
    
    
    
handle(1)
handle(2)